In [ ]:
!pip install transformers tensorflow pandas scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import shutil
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from google.colab import files

In [ ]:
df = pd.read_csv("dataset_ic.csv")
df

,topic,intent
0,vitamin untuk alergi kulit merah dan gatal yan...,0
1,obat untuk mengatasi gatalgatal pada anak,0
2,kulit hidung gatal hingga memerah apa penyebabnya,0
3,apa obat batuk alergi tenggorokan gatal,0
4,wajah merah dan gatal menandakan gejala apa,0
...,...,...
13441,cara mengatasi keputihan disertai bercak darah...,2
13442,haid sedikit setelah bulan melahirkan,2
13443,manfaat dan pengaruh mengonsumsi suplemen pena...,2
13444,pengobatan dan pengaruh dari operasi kista ova...,2


In [ ]:
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
sentences = df['topic'].tolist()
labels = df['intent'].tolist()

In [ ]:
def encode_texts(texts, tokenizer, max_length=64):
  encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="tf")
  return encodings['input_ids'], encodings['attention_mask']

In [ ]:
input_ids, attention_masks = encode_texts(sentences, tokenizer)

In [ ]:
labels = tf.convert_to_tensor(labels)

In [ ]:
input_ids_np = input_ids.numpy()
attention_masks_np = attention_masks.numpy()
labels_np = labels.numpy()

In [ ]:
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids_np, attention_masks_np, labels_np, test_size=0.2, random_state=42
)

In [ ]:
train_ids = tf.convert_to_tensor(train_ids)
val_ids = tf.convert_to_tensor(val_ids)
train_masks = tf.convert_to_tensor(train_masks)
val_masks = tf.convert_to_tensor(val_masks)
train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(({
    "input_ids":train_ids,
    "attention_mask": train_masks
  }, train_labels))

In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': val_ids, 'attention_mask': val_masks}, val_labels))

In [ ]:
batch_size=8
train_dataset = train_dataset.shuffle(100).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

patience = 3
min_delta = 1e-4
best_val_loss = float('inf')
wait = 0

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_loss_total = 0
    train_steps = 0

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True).logits
            loss_value = loss_fn(y_batch_train, logits)

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        train_loss_total += loss_value
        train_steps +=1

        if step % 50 == 0:
            print(f"Training loss at step {step}: {loss_value:.4f}")

    train_loss_avg = train_loss_total / train_steps


    val_loss_total = 0
    val_steps = 0
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False).logits
        val_loss_total += loss_fn(y_batch_val, val_logits)
        val_steps += 1

    val_loss_avg = val_loss_total / len(val_dataset)

    print("====================================================")
    print(f"Training loss: {train_loss_avg: .4f}")
    print(f"Validaiton loss: {val_loss_avg: .4f}")
    print("====================================================")

    if val_loss_avg < best_val_loss - min_delta:
      best_val_loss = val_loss_avg
      wait = 0
      print("Validation loss improved")
    else:
      wait +=1
      print(f"No improvement in validation loss. Patience: {wait}/{patience}")

    if wait >= patience:
      print("Early stoping triggered, Stopping training...")
      break

Epoch 1/10
Training loss at step 0: 1.0836
Training loss at step 50: 0.1045
Training loss at step 100: 0.2176
Training loss at step 150: 0.1952
Training loss at step 200: 0.2870
Training loss at step 250: 0.0481
Training loss at step 300: 0.1242
Training loss at step 350: 0.0192
Training loss at step 400: 0.1088
Training loss at step 450: 0.0246
Training loss at step 500: 0.4811
Training loss at step 550: 0.1707
Training loss at step 600: 0.8012
Training loss at step 650: 0.6713
Training loss at step 700: 0.3371
Training loss at step 750: 0.0180
Training loss at step 800: 0.0406
Training loss at step 850: 0.1180
Training loss at step 900: 0.0338
Training loss at step 950: 0.0777
Training loss at step 1000: 0.0629
Training loss at step 1050: 0.7523
Training loss at step 1100: 0.1115
Training loss at step 1150: 0.0119
Training loss at step 1200: 0.0191
Training loss at step 1250: 0.0106
Training loss at step 1300: 0.0227
Training loss:  0.2053
Validaiton loss:  0.1927
Validation loss imp

In [ ]:
preds = model.predict(val_dataset)
pred_labels = tf.argmax(preds.logits, axis=1).numpy()

true_labels = []
for _, label in val_dataset:
    true_labels.extend(label.numpy())
true_labels = np.array(true_labels)

target_names = ['Alergi', 'Obat', 'Menstruasi']
print(classification_report(true_labels, pred_labels, target_names=target_names))


337/337 [==============================] - 17s 32ms/step
              precision    recall  f1-score   support

      Alergi       0.94      0.95      0.94       913
        Obat       0.94      0.90      0.92       870
  Menstruasi       0.95      0.98      0.97       907

    accuracy                           0.94      2690
   macro avg       0.94      0.94      0.94      2690
weighted avg       0.94      0.94      0.94      2690



In [ ]:
test_text = "salep yang cocok untuk alergi "

test_encodings = tokenizer(test_text, truncation=True, padding=True, max_length=128, return_tensors="tf")

preds = model(test_encodings)
pred_label = tf.argmax(preds.logits, axis=1).numpy()[0]

label_mapping = {0: "alergi", 1: "obat", 2: "menstruasi"}
print(f"Predicted label: {label_mapping[pred_label]}")


Predicted label: alergi


In [ ]:
model.save_pretrained('fine_tuned_indobert')
tokenizer.save_pretrained('fine_tuned_indobert')

('fine_tuned_indobert/tokenizer_config.json',
 'fine_tuned_indobert/special_tokens_map.json',
 'fine_tuned_indobert/vocab.txt',
 'fine_tuned_indobert/added_tokens.json',
 'fine_tuned_indobert/tokenizer.json')

In [ ]:
shutil.make_archive('fine_tuned_indobert', 'zip', 'fine_tuned_indobert')
files.download('fine_tuned_indobert.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>